<p><font size="6"><b>Modificación de Datos (Reshaping)</b></font></p>

> *© 2016, Joris Van den Bossche and Stijn Van Hoey  (<mailto:jorisvandenbossche@gmail.com>, <mailto:stijnvanhoey@gmail.com>). Licensed under [CC BY 4.0 Creative Commons](http://creativecommons.org/licenses/by/4.0/)*

---

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Combinación de Datos

En Pandas, podemos combinar tablas de acuerdo al valor de una o varias `llaves` que son utilizadas para identificar filas, como un úndice. Por ejemplo:

In [2]:
df1 = pd.DataFrame(dict(id=range(4), age=np.random.randint(18, 31, size=4)))
df2 = pd.DataFrame(dict(id=list(range(3))+list(range(3)), 
                        score=np.random.random(size=6)))

df1

,id,age
0,0,25
1,1,28
2,2,29
3,3,21


In [3]:
df2

,id,score
0,0,0.569972
1,1,0.584212
2,2,0.959518
3,0,0.489202
4,1,0.476968
5,2,0.855301


In [4]:
pd.merge(df1, df2)

,id,age,score
0,0,25,0.569972
1,0,25,0.489202
2,1,28,0.584212
3,1,28,0.476968
4,2,29,0.959518
5,2,29,0.855301


Nótese que sin la información acerca de cual columna usar como llave, Pandas hizo lo correcto e identificó la columna `id` en ambas tablas como el punto común. A menos de que se especifique otra cosa, `merge` utiliza cualquier conjunto de columnas que las tablas tengan en común para combinar las tablas.

Nótese también que el `id=3` de `df1` se encuentra omitido de la tabla combinada. Esto es porque, por defecto, `merge` efectúa un **inner join** sobre las tablas, lo cual significa que el resultado equivale a la intersección de ambas.

In [5]:
pd.merge(df1, df2, how='outer')

,id,age,score
0,0,25,0.569972
1,0,25,0.489202
2,1,28,0.584212
3,1,28,0.476968
4,2,29,0.959518
5,2,29,0.855301
6,3,21,NaN


Como se ve arriba, el **outer join** resulta en la unión de ambas tablas, por lo que todas las filas van a ser representadas, con valores faltantes (NaN) insertados donde no se tenfa información. Es también posible utilizar **right** o **left** joins para incluir todas las filas de la tabla derecha o izquierda (el primer y segundo argumento de la función) pero no necesariamente de la otra.

# Pivoteo de Datos

## Cfr. excel

En Excel, la función de **Pivot** es conocida por su flexibilidad:

![](img/pivot_excel.png)

Los datos de la tabla:

In [6]:
excelample = pd.DataFrame({'Month': ["January", "January", "January", "January", 
                                  "February", "February", "February", "February", 
                                  "March", "March", "March", "March"],
                   'Category': ["Transportation", "Grocery", "Household", "Entertainment",
                                "Transportation", "Grocery", "Household", "Entertainment",
                                "Transportation", "Grocery", "Household", "Entertainment"],
                   'Amount': [74., 235., 175., 100., 115., 240., 225., 125., 90., 260., 200., 120.]})

In [7]:
excelample

,Month,Category,Amount
0,January,Transportation,74.0
1,January,Grocery,235.0
2,January,Household,175.0
3,January,Entertainment,100.0
4,February,Transportation,115.0
5,February,Grocery,240.0
6,February,Household,225.0
7,February,Entertainment,125.0
8,March,Transportation,90.0
9,March,Grocery,260.0


In [8]:
excelample_pivot = excelample.pivot(index="Category", columns="Month", values="Amount")
excelample_pivot

Month,February,January,March
Category,,,
Entertainment,125.0,100.0,120.0
Grocery,240.0,235.0,260.0
Household,225.0,175.0,200.0
Transportation,115.0,74.0,90.0


¿Interesado en los totales?

In [9]:
# sum columns
excelample_pivot.sum(axis=1)

Category
Entertainment     345.0
Grocery           735.0
Household         600.0
Transportation    279.0
dtype: float64

In [10]:
# sum rows
excelample_pivot.sum(axis=0)

Month
February    705.0
January     584.0
March       670.0
dtype: float64

## Pivoteo equivale a reordenar datos

Un subconjunto de muestras de los datos del Titanic:

In [11]:
df = pd.DataFrame({'Fare': [7.25, 71.2833, 51.8625, 30.0708, 7.8542, 13.0],
                   'Pclass': [3, 1, 1, 2, 3, 2],
                   'Sex': ['male', 'female', 'male', 'female', 'female', 'male'],
                   'Survived': [0, 1, 0, 1, 0, 1]})

In [12]:
df

,Fare,Pclass,Sex,Survived
0,7.2500,3,male,0
1,71.2833,1,female,1
2,51.8625,1,male,0
3,30.0708,2,female,1
4,7.8542,3,female,0
5,13.0000,2,male,1


In [13]:
df.pivot(index='Pclass', columns='Sex', values='Fare')

Sex,female,male
Pclass,,
1,71.2833,51.8625
2,30.0708,13.0000
3,7.8542,7.2500


In [14]:
df.pivot(index='Pclass', columns='Sex', values='Survived')

Sex,female,male
Pclass,,
1,1,0
2,1,1
3,0,0


Hasta ahora vamos bien...

Usemos ahora el set de datos completo:

In [15]:
df = pd.read_csv("data/titanic.csv")

In [16]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


E intentemos aplicar el mismo pivoteo (*las sentencias try-except se encuentran acá para evitar un long error*):

In [17]:
try:
    df.pivot(index='Sex', columns='Pclass', values='Fare')
except Exception as e:
    print("Exception!", e)

Exception! Index contains duplicate entries, cannot reshape


Esto no funciona, porque la tabla resultado terminaría con múltiples valores para una celda, como el error dice: `duplicated` values for the columns in the selection. Como un ejemplo, se pueden considerar las filas de las siguientes tres columnas de interés:

In [18]:
df.loc[[1, 3], ["Sex", 'Pclass', 'Fare']]

,Sex,Pclass,Fare
1,female,1,71.2833
3,female,1,53.1000


Ya que `pivot` va a reestructurar los datos, ¿dondé necesitaría ir ambos valores de la columna `fare` para la combinacion de `sex` y `pclass`?

Estos valores deben ser combinados de acuerdo a una función de agregación, la cual es soportado por la función `pivot_table`.

<div class="alert alert-danger">

<b>NOTA</b>:

 <ul>
  <li>**Pivot** es puramente para reestructuración: es necesario que solamente haya un valor por combinación de indice y columna.</li>
</ul>
</div>

# Pivoteo de Tablas - Agregación

In [19]:
df = pd.read_csv("data/titanic.csv")

In [20]:
df.pivot_table(index='Sex', columns='Pclass', values='Fare')

Pclass,1,2,3
Sex,,,
female,106.125798,21.970121,16.118810
male,67.226127,19.741782,12.661633


<div class="alert alert-info">

<b>RECORDAR</b>:

 <ul>
  <li>Por defecto, `pivot_table` toma el valor **medio** de todos los valores que terminarían en una celda. Sin embargo, puede especificarse otra función de agregación por medio del atributo `aggfunc`.</li>
</ul>
</div>

In [21]:
df.pivot_table(index='Sex', columns='Pclass', 
               values='Fare', aggfunc='max')

Pclass,1,2,3
Sex,,,
female,512.3292,65.0,69.55
male,512.3292,73.5,69.55


In [22]:
df.pivot_table(index='Sex', columns='Pclass', 
               values='Fare', aggfunc='count')

Pclass,1,2,3
Sex,,,
female,94,76,144
male,122,108,347


<div class="alert alert-info">

<b>RECORDAR</b>:

 <ul>
  <li>Existe una función atajo para una ejecución de `pivot_table` con `aggfunc=count` como función de agregación: `crosstab`</li>
</ul>
</div>

In [23]:
pd.crosstab(index=df['Sex'], columns=df['Pclass'])

Pclass,1,2,3
Sex,,,
female,94,76,144
male,122,108,347


<div class="alert alert-success">

<b>EJERCICIO</b>:

 <ul>
  <li>Construya una tabla pivote con las proporciones de supervivencia (número de sobrevivientes / número total de personas) para Pclass vs Sex.</li>
  <li>Ilustre el resultado como un gráfico de barras.</li>
</ul>
</div>

In [24]:
# %load snippets/06 - Modificacion de Datos (Reshaping)20.py

In [25]:
# %load snippets/06 - Modificacion de Datos (Reshaping)21.py

<div class="alert alert-success">

<b>EJERCICIO</b>:

 <ul>
  <li>Construya una tabla del Fare(precio de tiquete) medio pagado por personas menores/mayores de edad vs género</li>
</ul>
</div>

In [26]:
# %load snippets/06 - Modificacion de Datos (Reshaping)22.py

In [27]:
# %load snippets/06 - Modificacion de Datos (Reshaping)23.py

# Melt

La función `melt` ejecuta la operación inversa a `pivot`. Esto puede ser usado para hacer el DataFrame más largo (crear una version más clara de los datos).

In [28]:
pivoted = df.pivot_table(index='Sex', columns='Pclass', values='Fare').reset_index()
pivoted.columns.name = None

In [29]:
pivoted

,Sex,1,2,3
0,female,106.125798,21.970121,16.118810
1,male,67.226127,19.741782,12.661633


Asumamos que tenemos un DataFrame como el de arriba. Las observaciones (el promedio de tiquete pagado) se encuentran distribuidas en varias columnas. En un dataset claro, cada observación se encuentra en una columna. Para obtener esto, usamos `melt`.

In [30]:
pd.melt(pivoted)

,variable,value
0,Sex,female
1,Sex,male
2,1,106.126
3,1,67.2261
4,2,21.9701
5,2,19.7418
6,3,16.1188
7,3,12.6616


Como se observa, la función `melt` posiciona todas las etiquetas de columnas en una sola, y todos los valores en una segunda columna. 

En este caso, esto no es del todo lo que queremos. Podemos entonces mantener la columna de `Sex` separada:

In [31]:
pd.melt(pivoted, id_vars=['Sex']) #, var_name='Pclass', value_name='Fare')

,Sex,variable,value
0,female,1,106.125798
1,male,1,67.226127
2,female,2,21.970121
3,male,2,19.741782
4,female,3,16.118810
5,male,3,12.661633


# Cambiando de forma con `stack` y `unstack`

De la documentación:

> Pivotea un nivel de las (posiblemente jerárquicas) etiquetas de columnas,
retornando un DataFrame (o Series en el caso de un objeto con un único nivel
de etiquetas de columna) que tenga un índice jerárquico con un nuevo nivel interno
de etiquetas de filas.

Indeed... 
<img src="img/schema-stack.svg" width=50%>

Antes de definir `índice jerárquico`, podemos practicarlo con el siguiente ejemplo:

In [32]:
df = pd.DataFrame({'A':['one', 'one', 'two', 'two'], 
                   'B':['a', 'b', 'a', 'b'], 
                   'C':range(4)})
df

,A,B,C
0,one,a,0
1,one,b,1
2,two,a,2
3,two,b,3


Para usar `stack`/`unstack`, necesitamos los valores que queremos mover de filas a columnas (o viceversa) como un índice:

In [33]:
df = df.set_index(['A', 'B']) # Sí, se pueden combinar índices!
df

C
A   B   
one a  0
    b  1
two a  2
    b  3

In [34]:
result = df['C'].unstack()
result

B,a,b
A,,
one,0,1
two,2,3


In [35]:
df = result.stack().reset_index(name='C')
df

,A,B,C
0,one,a,0
1,one,b,1
2,two,a,2
3,two,b,3


<div class="alert alert-info">

<b>RECORDAR</b>:

 <ul>
  <li>**stack**: hace los datos *más largos* y *pequeños* </li>
  <li>**unstack**: hace los datos *más cortos* y *anchos* </li>
</ul>
</div>

## Mímica de pivote de tablas 

Para entender mejor y razonar acerca de pivoteo de tablas, podemos expresar esta función como una combinación de pasos más basicos. En corto, el pivoteo es una manera conveniente de describir una combinación de `groupby` y `stack`/`unstack`.

In [36]:
df = pd.read_csv("data/titanic.csv")

In [37]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [38]:
df.pivot_table(index='Pclass', columns='Sex', 
               values='Survived', aggfunc='mean')

Sex,female,male
Pclass,,
1,0.968085,0.368852
2,0.921053,0.157407
3,0.500000,0.135447


<div class="alert alert-success">

<b>EJERCICIO</b>:

 <ul>
  <li>Obtener el mismo resultado que arriba usando una combinacíon de `groupby` y `unstack`</li>
  <li>Primero, se usa `groupby` para calcular la proporción de supervivencia para todos los grupos</li>
  <li>Luego, se usa `unstack` para reformar el output de la operación de groupby.</li>
</ul>
</div>

In [39]:
# %load snippets/06 - Modificacion de Datos (Reshaping)37.py

## Mímica de melt

Como en el pivoteo anterior, se puede obtener el resultado de `melt` con `stack`/`unstack`.

Usando la misma tabla anterior, observemos el resultado de melt:

In [40]:
pivoted = df.pivot_table(index='Sex', columns='Pclass', values='Fare').reset_index()
pivoted.columns.name = None
pivoted

,Sex,1,2,3
0,female,106.125798,21.970121,16.118810
1,male,67.226127,19.741782,12.661633


In [41]:
pd.melt(pivoted, id_vars=['Sex'], var_name='Pclass', value_name='Fare')

,Sex,Pclass,Fare
0,female,1,106.125798
1,male,1,67.226127
2,female,2,21.970121
3,male,2,19.741782
4,female,3,16.118810
5,male,3,12.661633


<div class="alert alert-success">

<b>EJERCICIO</b>:

 <ul>
  <li>Obtener el mismo resultado que arriba usando `stack`/`unstack` (combinado con `set_index` / `reset_index`)</li>
  <li>Tip: fije las columnas como el índice some el cual no quiere aplicar stack.</li>
</ul>
</div>

In [42]:
# %load snippets/06 - Modificacion de Datos (Reshaping)40.py

In [43]:
# %load snippets/06 - Modificacion de Datos (Reshaping)41.py

In [44]:
# %load snippets/06 - Modificacion de Datos (Reshaping)42.py

In [45]:
# %load snippets/06 - Modificacion de Datos (Reshaping)43.py

# Ejercicios: Usando los métodos vistos sobre datos de películas

Estos ejercicios se basan en el [tutorial de Pycon de Brandon Rhodes](https://github.com/brandon-rhodes/pycon-pandas-tutorial/) (so credit to him!) (todo el crédito para él!) y los datasets que preparó para el mismo. Estos pueden ser descargados desde acá: [`titles.csv`](https://drive.google.com/open?id=0B3G70MlBnCgKajNMa1pfSzN6Q3M) y [`cast.csv`](https://drive.google.com/open?id=0B3G70MlBnCgKal9UYTJSR2ZhSW8) y puestos en el directorio `/data`.

In [46]:
cast = pd.read_csv('data/cast.csv')
cast.head()

FileNotFoundError: File b'data/cast.csv' does not exist

In [ ]:
titles = pd.read_csv('data/titles.csv')
titles.head()

<div class="alert alert-success">

<b>EJERCICIO</b>:

 <ul>
  <li>Grafique el número de roles para actores por año y el numero de roles para actrices por año durante todo el periodo representado por los datos recuperados.</li>
</ul>
</div>

In [ ]:
# %load snippets/06 - Modificacion de Datos (Reshaping)46.py

In [ ]:
# %load snippets/06 - Modificacion de Datos (Reshaping)47.py

In [ ]:
# %load snippets/06 - Modificacion de Datos (Reshaping)48.py

<div class="alert alert-success">

<b>EJERCICIO</b>:

 <ul>
  <li>Grafique el número de roles de actor y roles de actriz cada año usando kind='area' como tipo de gráfico.</li>
</ul>
</div>

In [ ]:
# %load snippets/06 - Modificacion de Datos (Reshaping)49.py

<div class="alert alert-success">

<b>EJERCICIO</b>:

 <ul>
  <li>Grafique la fracción de roles que han sido roles para actor en todo el periodo disponible en los datos.</li>
</ul>
</div>

In [ ]:
# %load snippets/06 - Modificacion de Datos (Reshaping)50.py

<div class="alert alert-success">

<b>EJERCICIO</b>:

 <ul>
  <li>Se define un año como "Superman year" cuando películas de ese año contienen más apariciones de Superman que de Batman. ¿Cuantos años en los datos provistos han sido años de Superman?</li>
</ul>
</div>

In [ ]:
# %load snippets/06 - Modificacion de Datos (Reshaping)51.py

In [ ]:
# %load snippets/06 - Modificacion de Datos (Reshaping)52.py

# Lidiando con Datos Duplicados

Es posible identificar y remover valores duplicados de un objeto de tipo `DataFrame`. Por ejemplo, el siguiente set de datos de información de barcos contiene filas duplicadas para el mismo nombre de barcos. Así podemos eliminar estos datos:

In [ ]:
vessels = pd.read_csv("data/vessel_information.csv", index_col='mmsi')
vessels.head()

In [ ]:
vessels.duplicated(subset='names')

In [ ]:
vessels.drop_duplicates(['names'])

### Reemplazo de Valores

Frecuentemene, puede desearse que columnas que se encuentran formateadas como Strings puedan ser representadas por números para el propósito de incluirlas en análisis cuantitativos. Por ejemplo, considere la variable de tratamiento en el set de datos de dystonia cervical:

In [ ]:
cdystonia = pd.read_csv("data/cdystonia.csv", index_col=None)
cdystonia.head()

In [ ]:
cdystonia.treat.value_counts()

Una manera lógica de especificar estos valores numéricamente es cambiarlos a valores enteros, usando "Placebo" como un valor base. Si creamos un diccionario con los valores originales como llaves y los valores de reemplazo como valores, puede usarse el método `map` para implementar los cambios:

In [ ]:
treatment_map = {'Placebo': 0, '5000U': 1, '10000U': 2}

In [ ]:
cdystonia['treatment'] = cdystonia.treat.map(treatment_map)
cdystonia.treatment

Alternativamente, si simplemente se desea reemplazar valores particulares en un objeto Series o Dataframe, puede emplearse el método de `replace`. 

Un ejemplo donde el reemplazo es útil es cuando se desea lidiar con ceros para varias transformaciones. Por ejemplo, si intentamos obtener el logaritmo para un conjunto de datos:

In [ ]:
vals = pd.Series([float(i)**10 for i in range(10)])
vals

In [ ]:
np.log(vals)

En situaciones como esta, se puede reemplazar el cero con un valor muy pequeño que no afecte el análisis posterior usando el método de `replace`:

In [ ]:
vals = vals.replace(0, 1e-6)
np.log(vals)

### Variables Indicadoras

Para algunos análisis estadísticos (Modelos de regresión o análisis de varianza), variables categóricas deben ser convertidas en columnas de indicadores, zeros o unos, para crear una **design matrix**. La función de Pandas `get_dummies` (las variables indicadoras también son conocidas como *variables dummy*) facilitan esta transformación.

Consideremos el siguiente DataFrame conteniendo los barcos que corresponden a los segmentos de tránsito en la costa este. El `tipo` denota la clase de bote; es posible crear una matriz de indicadores para esto. Por simplicidad, se filtran los 5 tipos más comunes de bote:

In [ ]:
top5 = vessels.type.isin(vessels.type.value_counts().index[:5])
top5.head(10)

In [ ]:
vessels5 = vessels[top5]

In [ ]:
pd.get_dummies(vessels5.type).head(10)

## Datos Categóricos

Pandas provee un `dtype` conveniente para representar datos categóricos, denominado `category`.

Por ejemplo, la columna `treat` en el dataset médico usado anteriormente representa tres niveles de tratamiento en una prueba clínica, y es importada por defecto como un tipo `object`, al ser una mezcla de caracteres String.

In [ ]:
cdystonia.treat.head()

Es posible convertir esto a una `category` por medio del constructor `Categorical` o usando casting del tipo de columna con `astype`:

In [ ]:
pd.Categorical(cdystonia.treat)

In [ ]:
cdystonia['treat'] = cdystonia.treat.astype('category')

In [ ]:
cdystonia.treat.describe()

Por defecto, una categoría no se encuentra ordenada.

In [ ]:
cdystonia.treat.cat.categories

Sin embargo, un orden puede ser impuesto. Por defecto el orden es léxico, pero va a asumir el orden de las categorias listadas como el deseado.

In [ ]:
cdystonia.treat.cat.categories = ['Placebo', '5000U', '10000U']

In [ ]:
cdystonia.treat.cat.as_ordered().head()